In [105]:
import numpy as np
from math import exp
from statsmodels.api import Logit

# Exercise II
# Defining the Logit Regression Model class
class LogitRegression:
    def __init__(self, y, X, β):
        self.X, self.y, self.β = X, y, β
        self.n, self.k = X.shape

    # Logistic function (sigmoid function)
    def logistic_function(self, z):
        return 1 / (1 + np.exp(-z))

    def μ(self):
        # Applying logistic function to Xβ
        return self.logistic_function(self.X @ self.β)
        
    # Finding log-likelihood function
    def logL(self):
        y = self.y
        μ = self.μ()
        return np.sum(y * np.log(μ) + (1 - y) * np.log(1 - μ))
    
    # Finding Gradient (first derivative of log-likelihood function)
    def G(self):
        y = self.y
        μ = self.μ()
        return self.X.T @ (y - μ)
    
    # Finding Hessian (second derivative of log-likelihood function)
    def H(self):
        X = self.X
        μ = self.μ()
        # W represents a diagonal matrix of weights, where each diagonal element is μ_i * (1 - μ_i)
        W = np.diag((μ * (1 - μ)).flatten())
        return -(X.T @ W @ X)

# Exercise 3
# Applying Newton-Raphson algorithm to find the MLE
def newton_raphson(model, tol=1e-6, max_iter=1000, display=True):
    i = 0
    error = 100  # Initial error value

    # Printing header of output
    if display:
        header = f'{"Iteration_k":<13}{"Log-likelihood":<16}{"β":<60}'
        print(header)
        print("-" * len(header))

    # While loop runs while any value in error is greater
    # than the tolerance until max iterations are reached
    while np.any(error > tol) and i < max_iter:
        H, G = model.H(), model.G()
        β_new = model.β - (np.linalg.inv(H) @ G)
        error = np.abs(β_new - model.β)
        model.β = β_new

        # Print iterations
        if display:
            β_list = [f'{t:.3}' for t in list(model.β.flatten())]
            update = f'{i:<13}{model.logL():<16.8}{β_list}'
            print(update)

        i += 1

    print(f'Number of iterations: {i}')
    print(f'β_hat = {model.β.flatten()}')

    # Returning a flat array for β 
    return model.β.flatten()

# Dataset
X = np.array([[1, 2, 4],
              [1, 1, 1],
              [1, 4, 3],
              [1, 5, 6],
              [1, 3, 5]])

y = np.array([1, 0, 1, 1, 0])

# Initial guess for β
init_β = np.array([0.1, 0.1, 0.1])

# Create an object with the logit model values
logit_model = LogitRegression(y, X, init_β)

# Use the Newton-Raphson method to find the MLE
β_hat = newton_raphson(logit_model, display=True)

# Verify with statsmodels
print(Logit(y, X).fit().summary())

Iteration_k  Log-likelihood  β                                                           
-----------------------------------------------------------------------------------------
0            -2.4304503      ['-2.11', '1.07', '-0.161']
1            -2.4091638      ['-2.39', '1.2', '-0.153']
2            -2.408845       ['-2.42', '1.23', '-0.158']
3            -2.4088448      ['-2.43', '1.23', '-0.158']
4            -2.4088448      ['-2.43', '1.23', '-0.158']
Number of iterations: 5
β_hat = [-2.4250679   1.22951194 -0.15811793]
Optimization terminated successfully.
         Current function value: 0.481769
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                    5
Model:                          Logit   Df Residuals:                        2
Method:                           MLE   Df Model:                            2
Date:                Thu, 17 Oct 2024   Pseudo R